In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

In [71]:
## Connect to spotify
CLIENT_ID = 'f4b0975d0a7e419fb4ef187ddd061914'
CLIENT_SECRET = '54314e98d08d4581be9ed226e49dc251'
REDIRECT_URI = 'http://localhost/'
USERNAME = 'sgtpepurr'
SCOPES = [
    'user-modify-playback-state',
    'user-read-playback-state',
    'user-read-currently-playing',
    'user-follow-modify',
    'user-follow-read',
    'user-read-recently-played',
    'user-read-playback-position',
    'user-top-read',
    'playlist-read-collaborative',
    'playlist-modify-public',
    'playlist-read-private',
    'playlist-modify-private',
    'app-remote-control',
    'streaming',
    'user-library-modify',
    'user-library-read'
]
token = spotipy.util.prompt_for_user_token('sgtpepurr',SCOPES,client_id=CLIENT_ID,client_secret=CLIENT_SECRET,redirect_uri=REDIRECT_URI)
sp = spotipy.Spotify(auth=token)

In [69]:
# read rym data
rym_df = pd.read_csv('../data/rym-1950-2021.csv', sep='~')
rym_df.tail(10)

,Unnamed: 0.1,Unnamed: 0,Rank,Artist,Album,Date,Genres,RYM Rating,Ratings,Reviews
2470,950,30,31,Nas,Magic,24 December 2021,"East Coast Hip Hop, Boom Bap",3.67,3041.0,39.0
2471,951,31,32,Effluence,Psychocephalic Spawning,13 December 2021,"Avant-Garde Metal, Brutal Death Metal, Technic...",3.63,480.0,11.0
2472,952,32,33,"岡部啓一 [Keiichi Okabe], 帆足圭吾 [Keigo Hoashi], 石濱翔...",NieR Replicant ver.1.22474487139...,21 April 2021,"Video Game Music, Cinematic Classical, Neoclas...",3.61,407.0,5.0
2473,953,33,34,Wild Up,Julius Eastman Vol. 1: Femenine,18 June 2021,"Minimalism, Chamber Music",3.67,1290.0,9.0
2474,954,34,35,Ad Nauseam,Imperative Imperceptible Impulse,12 February 2021,"Technical Death Metal, Avant-Garde Metal",3.63,2620.0,30.0
2475,955,35,36,DJ Sabrina the Teenage DJ,The Makin' Magick II Album,26 November 2021,Outsider House,3.65,1099.0,4.0
2476,956,36,37,STOMACH BOOK,STOMACH BOOK,8 October 2021,"Experimental Rock, Noise Pop, Emo",3.66,606.0,2.0
2477,957,37,38,Skee Mask,Pool,7 May 2021,"IDM, Breakbeat, Techno",3.64,2604.0,13.0
2478,958,38,39,McKinley Dixon,For My Mama and Anyone Who Look Like Her,7 May 2021,"Jazz Rap, Southern Hip Hop, Conscious Hip Hop",3.64,1773.0,21.0
2479,959,39,40,Jaubi,Nafs at Peace,28 May 2021,"Spiritual Jazz, Jazz Fusion",3.65,1965.0,9.0


In [74]:
# Get the most matching spotify album from the 

from collections import defaultdict

SIM_THRES = 0.8

def get_word_freq(l):
    d = defaultdict(int)
    for w in l:
        d[w] += 1
    return d

def freq_match(d1,d2):
    n = len(d1)
    sim = 0
    for k,v in d1.items():
        if k in d2 and d2[k] == v:
            sim += 1
    return sim/n


def are_same_albums(s1,s2):
    s1 = s1.lower()
    s2 = s2.lower()
    freq1 = get_word_freq(s1.split(' '))
    freq2 = get_word_freq(s2.split(' '))
    sim1 = freq_match(freq1, freq2)
    sim2 = freq_match(freq2, freq1)
    if max(sim1,sim2) >= SIM_THRES:
        return True
    return False

def get_spotify_album(name):
    search_res = sp.search(q=name, type='album')
    if are_same_albums(search_res['albums']['items'][0]['name'], name):
        return search_res['albums']['items'][0]['uri']
    else:
        print(search_res['albums']['items'][0]['name'])
        print(name)
        

def get_album_tracks(album_id):
    tracks = []
    for track in sp.album_tracks(album_id=album_id)['items']:
        tracks.append(track['uri'])
    return sp.tracks(tracks=tracks)

In [75]:
album_id = get_spotify_album('The Dark Side of the Moon')
tracks = get_album_tracks(album_id=album_id)

In [ ]:
for index, row in rym_df.iterrows():
    search_res = sp.search(q=row['Album'], type='album')
    search_res = search_res['albums']['items']